In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Reshape, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from tensorflow.keras.layers import Embedding, BatchNormalization, Concatenate
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dropout, ReLU
from tensorflow.keras.layers import Dense, Concatenate, Bidirectional, GRU, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, SpatialDropout1D, Dropout, concatenate

2024-06-06 23:07:32.044352: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 23:07:32.294531: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-06 23:07:33.283065: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-06-06 23:07:33.283771: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [2]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU'))) 

Num GPUs Available:  1


2024-06-06 23:07:34.902443: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-06 23:07:35.009115: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-06 23:07:35.009286: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
import os
import sys

# project_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)), '..')
project_dir = os.path.abspath(os.path.join(os.path.dirname(os.path.realpath('__file__')), '..'))
data_dir = os.path.join(project_dir, 'preprocessing')
sys.path.append(data_dir)

from preprocess import preprocess

In [4]:
from transformers import AutoTokenizer
# , AutoModelForSequenceClassification, Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification

# model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels = 3)
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base",use_fast=False)

/home/phannhaan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/phannhaan/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Evaluation

In [5]:
def evaluation(y_pred, y_test):
    # Tính toán các chỉ số đánh giá
    accuracy  = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-micro: {f1_micro}")
    print(f"F1-macro: {f1_macro}")

## Feature 

In [10]:
def tfidf_svd(X_train, X_test, X_dev):
    tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
    tfidf_vect.fit(X_train) # learn vocabulary and idf from training set
    X_train_tfidf =  tfidf_vect.transform(X_train)
    X_test_tfidf =  tfidf_vect.transform(X_test)
    X_dev_tfidf =  tfidf_vect.transform(X_dev)

    svd = TruncatedSVD(n_components=300, random_state=42)
    svd.fit(X_train_tfidf)

    X_train_tfidf_svd = svd.transform(X_train_tfidf)
    X_test_tfidf_svd = svd.transform(X_test_tfidf)
    X_dev_tfidf_svd = svd.transform(X_dev_tfidf)

    return X_train_tfidf_svd, X_test_tfidf_svd, X_dev_tfidf_svd

In [11]:
def word2vec(X_train, X_test, X_dev):
    tokenizer = Tokenizer(num_words=10000)
    tokenizer.fit_on_texts(X_train) 
                                
    sequences = tokenizer.texts_to_sequences(X_train)
    X_train_feature = pad_sequences(sequences, maxlen=300) 
    # tr_y = to_categorical(tr_label)

    sequences = tokenizer.texts_to_sequences(X_test)
    X_test_feature = pad_sequences(sequences, maxlen=300)
    # val_y = to_categorical(val_label)

    sequences = tokenizer.texts_to_sequences(X_dev)
    X_dev_feature = pad_sequences(sequences, maxlen=300)
    # ts_y = to_categorical(ts_label)

    return X_train_feature, X_test_feature, X_dev_feature

## Model

In [46]:
# model1
def create_lstm_model():
    input_layer = Input(shape=(300,))
    emb = Embedding(10000, 200, input_length=300, trainable=True)(input_layer)
    
    # layer = Reshape((10, 30))(input_layer)
    layer = Sequential()(emb)
    layer = LSTM(128, activation='relu', return_sequences=True)(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(512, activation='relu')(layer)
    layer = Dense(128, activation='relu')(layer)
    
    output_layer = Dense(10, activation='softmax')(layer)
    
    classifier = models.Model(input_layer, output_layer)
    
    classifier.compile(optimizer=optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

In [6]:
# 2
def create_lstm_model():
    input = Input(shape = (300,))
    emb = Embedding(10000, 200, input_length=300, trainable=True)(input)
    # emb = Reshape((10, 30))(input)
    x1 = SpatialDropout1D(0.2)(emb)

    x = Bidirectional(GRU(112, return_sequences = True))(x1)
    x = Conv1D(3, kernel_size = 3, padding = "valid", kernel_initializer = "he_uniform")(x)
        
    y = Bidirectional(LSTM(112, return_sequences = True))(x1)
    y = Conv1D(3, kernel_size = 3, padding = "valid", kernel_initializer = "he_uniform")(y)
        
    avg_pool1 = GlobalAveragePooling1D()(x)
    max_pool1 = GlobalMaxPooling1D()(x)
        
    avg_pool2 = GlobalAveragePooling1D()(y)
    max_pool2 = GlobalMaxPooling1D()(y)

    conc = Concatenate(axis=-1)([avg_pool1, max_pool1, avg_pool2, max_pool2])

    hid_layer = Dense(128, activation='relu')(conc)
    dropout = Dropout(0.3)(hid_layer)
    output_layer = Dense(10, activation='softmax')(dropout)
    
    classifier = models.Model(inputs=input, outputs=output_layer)
    
    classifier.compile(optimizer=optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

In [80]:
# 3
def create_lstm_model():
    input = Input(shape=(300,))
    emb = Embedding(10000, 200, input_length=300, trainable=True)(input)
    # emb = Reshape((10, 30))(input)

    branch1 = Sequential()(emb)
    branch1=LSTM(64, return_sequences=True)(branch1)
    branch1=BatchNormalization()(branch1)
    branch1=ReLU()(branch1)
    branch1=Dropout(0.5)(branch1)
    branch1=GlobalMaxPooling1D()(branch1)
    
        # Branch 2
    branch2 = Sequential()(emb)
    branch2=LSTM(64, return_sequences=True)(branch2)
    branch2=BatchNormalization()(branch2)
    branch2=ReLU()(branch2)
    branch2=Dropout(0.5)(branch2)
    branch2=GlobalMaxPooling1D()(branch2)

    concatenated = Concatenate()([branch1, branch2])

    hid_layer = Dense(128, activation='relu')(concatenated)
    dropout = Dropout(0.3)(hid_layer)
    output_layer = Dense(10, activation='softmax')(dropout)
    
    classifier = models.Model(inputs=input, outputs=output_layer)
    
    classifier.compile(optimizer=optimizers.legacy.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return classifier

## ViHSD

### load data

In [7]:
train = pd.read_csv("../data/ViHSD/train.csv")
test = pd.read_csv("../data/ViHSD/test.csv")
dev = pd.read_csv("../data/ViHSD/dev.csv")

In [31]:
train['label_id'].value_counts()

label_id
0    19886
1     4162
Name: count, dtype: int64

In [8]:
X_train = train["free_text"].copy()
y_train = train["label_id"].copy()

X_test = test["free_text"].copy()
y_test = test["label_id"].copy()

X_dev = dev["free_text"].copy()
y_dev = dev["label_id"].copy()

In [9]:
X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)
X_dev = X_dev.apply(preprocess)

### train

In [30]:
# X_train_feature, X_test_feature, X_dev_feature = tfidf_svd(X_train, X_test, X_dev)

In [16]:
# X_train_feature, X_test_feature, X_dev_feature = word2vec(X_train, X_test, X_dev)

In [10]:
X_train_feature = np.array(tokenizer(list(X_train), max_length=300, padding='max_length', truncation=True)['input_ids'])
X_test_feature = np.array(tokenizer(list(X_test), max_length=300, padding='max_length', truncation=True)['input_ids'])
X_dev_feature = np.array(tokenizer(list(X_dev), max_length=300, padding='max_length', truncation=True)['input_ids'])

In [11]:
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True,min_delta=0.001)
class_weights = {0: 1.0, 1: 4.0}

model_hsd = create_lstm_model()
history = model_hsd.fit(X_train_feature, y_train, 
                    validation_data=(X_dev_feature, y_dev), 
                    batch_size=256, epochs=100, verbose=True,
                    callbacks=[early_stopping],
                    class_weight=class_weights
                )

2024-06-06 23:07:53.422885: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-06 23:07:53.422989: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-06 23:07:53.423007: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-06 23:07:56.717127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-06 23:07:56.717535: I tensorflow/compile

Epoch 1/100


2024-06-06 23:08:04.905013: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8204
2024-06-06 23:08:15.024919: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


94/94 [==============================] - 36s 172ms/step - loss: 1.1610 - accuracy: 0.7233 - val_loss: 0.4391 - val_accuracy: 0.8099
Epoch 2/100
94/94 [==============================] - 15s 163ms/step - loss: 0.6327 - accuracy: 0.8326 - val_loss: 0.4582 - val_accuracy: 0.7915
Epoch 3/100
94/94 [==============================] - 15s 161ms/step - loss: 0.5513 - accuracy: 0.8542 - val_loss: 0.3864 - val_accuracy: 0.8368
Epoch 4/100
94/94 [==============================] - 15s 160ms/step - loss: 0.4973 - accuracy: 0.8644 - val_loss: 0.4184 - val_accuracy: 0.8155
Epoch 5/100
94/94 [==============================] - 15s 160ms/step - loss: 0.4501 - accuracy: 0.8792 - val_loss: 0.4509 - val_accuracy: 0.8073
Epoch 6/100
94/94 [==============================] - 15s 154ms/step - loss: 0.4054 - accuracy: 0.8891 - val_loss: 0.4427 - val_accuracy: 0.8245
Epoch 7/100
94/94 [==============================] - 15s 154ms/step - loss: 0.3709 - accuracy: 0.9003 - val_loss: 0.4783 - val_accuracy: 0.8125
Epoc

In [119]:
model_hsd.save('../models/model_lstm_hsd_2_phobert.h5')

In [12]:
model_eva = create_lstm_model()
# model_eva.load_weights('../models/model_lstm_hsd_2_w2v.h5')
model_eva= model_hsd

y_pred = model_eva.predict(X_test_feature)
y_pred_classes = np.argmax(y_pred, axis=1)

evaluation(y_pred_classes, y_test)

209/209 [==============================] - 10s 42ms/step
Accuracy: 0.8374251497005988
Precision: 0.5146871008939975
Recall: 0.7120141342756183
F1-micro: 0.8374251497005988
F1-macro: 0.7478112760322263


In [13]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score

# Tính toán các chỉ số đánh giá
f1_micro = f1_score(y_test, y_pred_classes, average='micro')
print("F1 - micro: " + str(f1_micro))

f1_macro = f1_score(y_test, y_pred_classes, average='macro')
print("F1 - macro: " + str(f1_macro))

accuracy = accuracy_score(y_test, y_pred_classes)
print("Accuracy: " + str(accuracy))

precision_macro = precision_score(y_test, y_pred_classes, average='macro')
print("Precision - macro: " + str(precision_macro))

recall_macro = recall_score(y_test, y_pred_classes, average='macro')
print("Recall - macro: " + str(recall_macro))

F1 - micro: 0.8374251497005988
F1 - macro: 0.7478112760322263
Accuracy: 0.8374251497005988
Precision - macro: 0.7254702614364394
Recall - macro: 0.7875139164528777


In [14]:
text ="Một  phút  bóc  đồng  bằng  cả  đời  bóc  cức  😂"

text = preprocess(text)

sequences = tokenizer(text, max_length=300, padding='max_length', truncation=True)['input_ids']

y_ =model_hsd.predict([sequences])
np.argmax(y_, axis=1)

1/1 [==============================] - 0s 283ms/step


array([1])

In [ ]:
# Lấy cái này
# '../models/model_lstm_hsd_2_w2v.h5'

# 2

# w2v

# Accuracy: 0.8670658682634731
# Precision: 0.5893118594436311
# Recall: 0.7111307420494699
# F1-micro: 0.8670658682634731
# F1-macro: 0.7813813562357896

# F1 - micro: 0.8670658682634731
# F1 - macro: 0.7813813562357896
# Accuracy: 0.8670658682634731
# Precision - macro: 0.7638881465076642
# Recall - macro: 0.8050066111112526

In [97]:
from collections import Counter

element_counts = Counter(y_test)
print(element_counts)

Counter({0: 5548, 1: 1132})


In [96]:
from collections import Counter

element_counts = Counter(y_pred_classes)
print(element_counts)

Counter({0: 5314, 1: 1366})


In [ ]:
# # 3

# Accuracy: 0.8591317365269461
# Precision: 0.5920925747348119
# Recall: 0.5424028268551236
# F1-micro: 0.8591317365269461
# F1-macro: 0.7410370473638934

# w2v

# Accuracy: 0.8565868263473054
# Precision: 0.5663109756097561
# Recall: 0.6563604240282686
# F1-micro: 0.8565868263473054
# F1-macro: 0.7601292776440715

In [ ]:
# model 1

# tf-idf
# Accuracy: 0.8494011976047904
# Precision: 0.5474397590361446
# Recall: 0.642226148409894
# F1-micro: 0.8494011976047903
# F1-macro: 0.7493816662937272

## CTSD

### load data

In [15]:
train = pd.read_csv("../data/ViCTSD/ViCTSD_train.csv")
test = pd.read_csv("../data/ViCTSD/ViCTSD_test.csv")
dev = pd.read_csv("../data/ViCTSD/ViCTSD_valid.csv")

In [16]:
# # Lọc các hàng có nhãn 1
# df_label_1 = train[train['Toxicity'] == 1]

# # Nhân đôi các hàng có nhãn 1
# df_label_1_doubled = pd.concat([df_label_1] * 3, ignore_index=True)

# # Kết hợp với DataFrame gốc
# train = pd.concat([train, df_label_1_doubled], ignore_index=True)

In [17]:
train["Toxicity"].value_counts()

Toxicity
0    6241
1     759
Name: count, dtype: int64

In [18]:
# # Lọc các hàng có nhãn 1
# df_label_1 = dev[dev['Toxicity'] == 1]

# # Nhân đôi các hàng có nhãn 1
# df_label_1_doubled = pd.concat([df_label_1] * 3, ignore_index=True)

# # Kết hợp với DataFrame gốc
# dev = pd.concat([dev, df_label_1_doubled], ignore_index=True)

In [19]:
dev["Toxicity"].value_counts()

Toxicity
0    1768
1     232
Name: count, dtype: int64

In [20]:
X_train = train["Comment"].copy()
y_train = train["Toxicity"].copy()

X_test = test["Comment"].copy()
y_test = test["Toxicity"].copy()

X_dev = dev["Comment"].copy()
y_dev = dev["Toxicity"].copy()

In [21]:
X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)
X_dev = X_dev.apply(preprocess)

In [22]:
# def augment_data(X, y, num_augmented_samples=10000):
#     augmented_X = []
#     augmented_y = []
#     for _ in range(num_augmented_samples):
#         idx = np.random.choice(len(X))
#         sample = X[idx]
#         label = y[idx]
#         # Thực hiện một số phép biến đổi ngẫu nhiên, ví dụ thay đổi một số từ trong mẫu
#         augmented_sample = sample.copy()
#         change_idx = np.random.choice(len(sample), size=5, replace=False)
#         augmented_sample[change_idx] = np.random.randint(1, 5000, size=5)
#         augmented_X.append(augmented_sample)
#         augmented_y.append(label)
#     return np.array(augmented_X), np.array(augmented_y)
# augmented_X, augmented_y = augment_data(X_train_feature, y_train)

### train

In [23]:
# X_train_feature, X_test_feature, X_dev_feature = tfidf_svd(X_train, X_test, X_dev)

In [24]:
# X_train_feature, X_test_feature, X_dev_feature = word2vec(X_train, X_test, X_dev)

In [25]:
X_train_feature = np.array(tokenizer(list(X_train), max_length=300, padding='max_length', truncation=True)['input_ids'])
X_test_feature = np.array(tokenizer(list(X_test), max_length=300, padding='max_length', truncation=True)['input_ids'])
X_dev_feature = np.array(tokenizer(list(X_dev), max_length=300, padding='max_length', truncation=True)['input_ids'])

In [26]:
# y_train_cate = to_categorical(y_train)
# y_test_cate = to_categorical(y_test)
# y_dev_cate = to_categorical(y_dev)

In [27]:
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3, restore_best_weights=True,min_delta=0.001)
class_weights = {0: 1.0, 1: 5.0}
model_ctsd = create_lstm_model()
with tf.device('/gpu:0'):
    history = model_ctsd.fit(X_train_feature, y_train, 
                        validation_data=(X_dev_feature, y_dev),
                        epochs=100, batch_size=256, verbose=True,
                        callbacks=[early_stopping],
                        class_weight=class_weights
                    )

Epoch 1/100
28/28 [==============================] - 11s 222ms/step - loss: 1.8395 - accuracy: 0.6690 - val_loss: 0.5108 - val_accuracy: 0.8840
Epoch 2/100
28/28 [==============================] - 5s 171ms/step - loss: 1.0796 - accuracy: 0.7394 - val_loss: 0.5114 - val_accuracy: 0.8840
Epoch 3/100
28/28 [==============================] - 5s 175ms/step - loss: 0.9194 - accuracy: 0.7903 - val_loss: 0.4011 - val_accuracy: 0.8670
Epoch 4/100
28/28 [==============================] - 5s 165ms/step - loss: 0.6467 - accuracy: 0.8526 - val_loss: 0.4754 - val_accuracy: 0.7730
Epoch 5/100
28/28 [==============================] - 5s 167ms/step - loss: 0.4331 - accuracy: 0.8869 - val_loss: 0.4898 - val_accuracy: 0.8050
Epoch 6/100
28/28 [==============================] - 5s 163ms/step - loss: 0.3011 - accuracy: 0.9197 - val_loss: 0.5334 - val_accuracy: 0.8095
Epoch 6: early stopping


In [37]:
model_ctsd.save('../models/model_lstm_ctsd_2_phobert.h5')

In [29]:
# y_pred = model_eva.predict([X_test_feature,X_test_feature])

In [30]:
model_eva = create_lstm_model()
# model_eva.load_weights('../models/model_lstm_ctsd_2_w2v.h5')
model_eva= model_ctsd

y_pred = model_eva.predict(X_test_feature)
y_pred_classes = np.argmax(y_pred, axis=-1)

evaluation(y_pred_classes, y_test)

32/32 [==============================] - 2s 44ms/step
Accuracy: 0.88
Precision: 0.43243243243243246
Recall: 0.2909090909090909
F1-micro: 0.88
F1-macro: 0.6408733959011683


In [31]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score

# Tính toán các chỉ số đánh giá
f1_micro = f1_score(y_test, y_pred_classes, average='micro')
print("F1 - micro: " + str(f1_micro))

f1_macro = f1_score(y_test, y_pred_classes, average='macro')
print("F1 - macro: " + str(f1_macro))

accuracy = accuracy_score(y_test, y_pred_classes)
print("Accuracy: " + str(accuracy))

precision_macro = precision_score(y_test, y_pred_classes, average='macro')
print("Precision - macro: " + str(precision_macro))

recall_macro = recall_score(y_test, y_pred_classes, average='macro')
print("Recall - macro: " + str(recall_macro))

F1 - micro: 0.88
F1 - macro: 0.6408733959011683
Accuracy: 0.88


Precision - macro: 0.6740995855466698
Recall - macro: 0.621859039836568


In [32]:
from collections import Counter

element_counts = Counter(y_test)
print(element_counts)

Counter({0: 890, 1: 110})


In [33]:
from collections import Counter

element_counts = Counter(y_pred_classes)
print(element_counts)

Counter({0: 926, 1: 74})


In [34]:
# Lấy cái này
# '../models/model_lstm_ctsd_2_w2v.h5'

# # 2

# Accuracy: 0.869
# Precision: 0.4198473282442748
# Recall: 0.5
# F1-micro: 0.869
# F1-macro: 0.6909787011197894

# F1 - micro: 0.869
# F1 - macro: 0.6909787011197894
# Accuracy: 0.869
# Precision - macro: 0.6782780945018843
# Recall - macro: 0.7073033707865168


In [35]:
# 1

# Accuracy: 0.879
# Precision: 0.39622641509433965
# Recall: 0.19090909090909092
# F1-micro: 0.879
# F1-macro: 0.5959002240916939


In [36]:
# # 3

# Accuracy: 0.886
# Precision: 0.47297297297297297
# Recall: 0.3181818181818182
# F1-micro: 0.886
# F1-macro: 0.65882972610611